In [3]:
import os
import pandas as pd
import numpy as np
import pywt
import itertools
import networkx as nx
import matplotlib.pyplot as plt


folder_path = 'C:\\Users\\19606\\Desktop\\GMDandGIC\\GICdata\\event_20131002\\GIC\\'

time_column = 'SampleDateTime'
gic_column = 'GICMeasured'

file_names = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

In [4]:
latitude_file_path = 'C:\\Users\\19606\\Desktop\\GMDandGIC\\GICdata\\event_20131002\\gic_monitors.csv' 
latitude_df = pd.read_csv(latitude_file_path)
latitude_dict = pd.Series(latitude_df['Latitude'].values, index=latitude_df['Device ID']).to_dict()
latitude_dict

{10056: 44.8,
 10075: 41.7,
 10094: 41.9,
 10222: 40.2,
 10224: 40.2,
 10299: 46.6,
 10300: 46.6,
 10301: 44.0,
 10318: 44.4,
 10319: 43.7,
 10320: 43.7,
 10321: 43.1,
 10322: 43.3,
 10323: 34.6,
 10397: 39.1,
 10398: 41.1,
 10077: 42.3,
 10083: 40.8,
 10112: 47.7,
 10115: 47.6,
 10116: 47.9,
 10119: 47.5,
 10121: 46.9,
 10257: 47.4,
 10258: 47.4,
 10259: 47.4,
 10260: 47.4,
 10261: 45.1,
 10262: 45.1,
 10263: 45.1,
 10264: 45.1,
 10331: 32.2,
 10360: 40.2,
 10361: 40.2,
 10362: 39.8,
 10363: 40.2,
 10364: 40.2,
 10399: 39.1,
 10400: 39.1,
 10401: 39.1,
 10402: 39.5,
 10403: 39.5,
 10404: 39.5,
 10405: 39.5,
 10406: 39.5,
 10407: 39.5,
 10408: 39.7,
 10409: 39.3,
 10410: 39.3,
 10411: 39.3,
 10412: 39.3,
 10413: 39.3,
 10425: 36.8,
 10437: 39.6,
 10180: 41.7,
 10181: 41.9,
 10183: 41.1,
 10184: 39.0,
 10185: 39.3,
 10187: 41.0,
 10193: 41.6,
 10197: 36.4,
 10198: 37.2,
 10199: 35.0,
 10202: 34.9,
 10213: 30.5,
 10214: 32.6,
 10215: 40.6,
 10216: 40.9,
 10218: 42.6,
 10326: 29.4,
 10376

In [5]:
def determine_max_level(data_length, wavelet='db4'):
    return pywt.swt_max_level(data_length)


def modwt(data, wavelet, level):
    return pywt.swt(data, wavelet, level=level, start_level=0)

def wavelet_cross_correlation(wt1, wt2):
    corrs = []
    for coeff1, coeff2 in zip(wt1, wt2):
        corr = np.corrcoef(coeff1[0], coeff2[0])[0, 1]
        corrs.append(corr)
    return corrs

def sliding_window_cross_correlation(wt1, wt2, window_size=30):

    assert len(wt1) == len(wt2)
    sliding_corrs = []
    
    for start in range(len(wt1) - window_size + 1):
        end = start + window_size
        window_corr = np.corrcoef(wt1[start:end], wt2[start:end])[0, 1]
        sliding_corrs.append(window_corr)
    
    return sliding_corrs

def make_length_even(data):
    return data if len(data) % 2 == 0 else np.append(data, data[-1])
edges = []

In [6]:
modwt_results = {}


for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    df[time_column] = pd.to_datetime(df[time_column])
    df = df.drop_duplicates(subset=time_column).set_index(time_column)
    df = df.reindex(pd.date_range(start=df.index.min(), end=df.index.max(), freq='T'), method='nearest')
    resampled_df = df.resample('T').mean().interpolate(method='spline', order=3)
    gic_signal = make_length_even(resampled_df[gic_column].values)
    level = determine_max_level(len(gic_signal))
    modwt_result = modwt(gic_signal, 'db4', level)
    modwt_results[file_name] = modwt_result

In [7]:
print("The wavelet transform coefficients of '2013E02_10056.csv' is:")
print(modwt_results['2013E02_10056.csv'])
print("The wavelet transform coefficients of '2013E02_10121.csv' is:")
print(modwt_results['2013E02_10121.csv'])

The wavelet transform coefficients of '2013E02_10056.csv' is:
[(array([1.71528357, 1.63627665, 1.49843081, ..., 1.54107532, 1.52978502,
       1.61506904]), array([ 0.01099079, -0.11197576,  0.10628366, ..., -0.04284867,
       -0.07861733,  0.13332257]))]
The wavelet transform coefficients of '2013E02_10121.csv' is:
[(array([2.2906629 , 2.15831847, 2.05275611, ..., 2.44732945, 2.29819186,
       2.29164274]), array([-0.09942156,  0.02058632,  0.03275968, ..., -0.10558983,
        0.02716292,  0.08095472]))]


In [8]:
desired_length = 1622  

filtered_modwt_results = {file: modwt_vals for file, modwt_vals in modwt_results.items() if len(modwt_vals[0][0]) == desired_length}

filtered_modwt_results

{'2013E02_10056.csv': [(array([1.71528357, 1.63627665, 1.49843081, ..., 1.54107532, 1.52978502,
          1.61506904]),
   array([ 0.01099079, -0.11197576,  0.10628366, ..., -0.04284867,
          -0.07861733,  0.13332257]))],
 '2013E02_10075.csv': [(array([-3.01563872, -1.62265178,  0.45297635, ..., -2.97403737,
          -1.71927223, -2.05354877]),
   array([ 0.17460861, -0.93217187, -0.1414064 , ...,  1.76989585,
          -0.4176934 , -0.36671653]))],
 '2013E02_10077.csv': [(array([-0.72924141, -0.809588  , -0.9006967 , ..., -0.75243032,
          -0.75633372, -0.73224205]),
   array([-0.00502364, -0.0001406 , -0.00460478, ..., -0.06298094,
           0.02133734,  0.02013054]))],
 '2013E02_10083.csv': [(array([-12.60782975, -12.50861742, -12.41943926, ..., -12.58283994,
          -12.57898828, -12.6086264 ]),
   array([ 0.01005597,  0.00963081,  0.00694375, ...,  0.07958165,
          -0.03188831, -0.03145638]))],
 '2013E02_10112.csv': [(array([-0.28284271, -0.28284271, -0.28284271

In [9]:
sliding_correlations = {}
for (file1, wt1), (file2, wt2) in itertools.combinations(filtered_modwt_results.items(), 2):
    corrs_over_time = sliding_window_cross_correlation(wt1[0][0], wt2[0][0])
    sliding_correlations[(file1, file2)] = corrs_over_time

c:\Users\19606\anaconda3\envs\py37\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\Users\19606\anaconda3\envs\py37\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [10]:
file_pair_to_visualize = (file_names[0], file_names[9])
correlations_over_time = sliding_correlations[file_pair_to_visualize]
time_axis = range(len(correlations_over_time))

In [11]:
correlation_threshold = 0.8

node_degrees = {file_name: [0] * len(time_axis) for file_name in file_names}

for (file1, file2), correlations in sliding_correlations.items():
    for t, corr in enumerate(correlations):
        if abs(corr) > correlation_threshold:
            node_degrees[file1][t] += 1
            node_degrees[file2][t] += 1

In [12]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection
import numpy as np

cmap = plt.cm.hot

max_correlations = {}
for node in node_degrees:
    max_correlations[node] = np.full(len(time_axis), -np.inf)

for (node1, node2), correlations in sliding_correlations.items():
    for i, corr in enumerate(correlations):
        max_correlations[node1][i] = max(max_correlations[node1][i], corr)
        max_correlations[node2][i] = max(max_correlations[node2][i], corr)
max_correlations


{'2013E02_10056.csv': array([0.43740107, 0.48726216, 0.64149565, ..., 0.56564331, 0.68925955,
        0.79250069]),
 '2013E02_10075.csv': array([0.43407544, 0.50683289, 0.56347665, ..., 0.40523249, 0.47613724,
        0.46357189]),
 '2013E02_10077.csv': array([0.91381808, 0.84014005, 0.84061952, ..., 0.81498636, 0.80774207,
        0.79420205]),
 '2013E02_10083.csv': array([0.86030272, 0.87932188, 0.88798283, ..., 0.61348171, 0.64964695,
        0.61931105]),
 '2013E02_10094.csv': array([-inf, -inf, -inf, ..., -inf, -inf, -inf]),
 '2013E02_10112.csv': array([0.54125755, 0.54787769, 0.55544084, ..., 0.52036292, 0.48980931,
        0.4305529 ]),
 '2013E02_10115.csv': array([0.91458626, 0.71486624, 0.82067356, ..., 0.67714194, 0.69952925,
        0.7146868 ]),
 '2013E02_10116.csv': array([0.68490452, 0.8053719 , 0.86275246, ..., 0.66734202, 0.68501514,
        0.67379456]),
 '2013E02_10119.csv': array([-inf, -inf, -inf, ..., -inf, -inf, -inf]),
 '2013E02_10121.csv': array([0.81694747, 0.8

In [22]:
node_degrees = {file_name: [0] * len(time_axis) for file_name in file_names}

for (file1, file2), correlations in sliding_correlations.items():
    for t, corr in enumerate(correlations):
        if abs(corr) > correlation_threshold:
            node_degrees[file1][t] += 1
            node_degrees[file2][t] += 1
node_degrees_1 = {file_name.split('_')[-1].split('.')[0]: degrees 
                        for file_name, degrees in node_degrees.items()}
node_degrees_1 = {int(key): value for key, value in node_degrees_1.items()}

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

cmap = plt.cm.viridis
norm = plt.Normalize(vmin=0, vmax=1)  

correlations_info = {node: {'values': [], 'exceeds_80_percentile': False} for node in node_degrees}

for (node1, node2), correlations in sliding_correlations.items():
    for i, corr in enumerate(correlations):
        if corr > 0.85:  # 只考虑超过0.85的相关性
            correlations_info[node1]['values'].append(corr)
            correlations_info[node2]['values'].append(corr)

# Determine which correlation values exceed the 80% threshold
for node, info in correlations_info.items():
    if info['values']: 
        sorted_corrs = sorted(info['values'], reverse=True)
        
        percentile_index = int(np.ceil(0.8 * len(sorted_corrs))) - 1
        percentile_value = sorted_corrs[percentile_index]
        
        info['exceeds_80_percentile'] = sorted_corrs[0] > percentile_value

# draw the circle and set the color
fig, ax = plt.subplots(figsize=(14, 6))


for node, degrees_over_time in node_degrees.items():
    lat = latitude_dict[int(node.split('_')[-1].split('.')[0])]
    for time_idx, degree in enumerate(degrees_over_time):
        if correlations_info[node]['exceeds_80_percentile']:
          
            color = cmap(norm(correlations_info[node]['values'][0]))
        else:
            color = 'white'  # If the threshold of 80% is not exceeded, it is set to white
        circle = Circle((time_idx, lat), degree * 0.01, color=color, ec='black', lw=0.1)
        ax.add_patch(circle)
        
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
fig.colorbar(sm, ax=ax)
ax.set_xlabel('Time Index')
ax.set_ylabel('Latitude')
ax.set_xlim(0, len(time_axis))
ax.set_ylim(min(latitude_dict.values()), max(latitude_dict.values()))

plt.savefig('C:/Users/19606/Desktop/GMDandGIC/plot5.png')  # Save the figure to a file
plt.close()


In [20]:
correlations_info

{'2013E02_10056.csv': {'values': [0.8575014280660619,
   0.8541969978922445,
   0.8870435238189351,
   0.9228046889405487,
   0.9031624343444818,
   0.8619623486372815,
   0.8787971727416264,
   0.8794583181396575,
   0.8792326143732381,
   0.8647190820187329,
   0.860270551441746,
   0.8742397322738591,
   0.8575662416464834,
   0.8783493322063819,
   0.8865095673818927,
   0.8806922897441604,
   0.880746149983317,
   0.8812089050697699,
   0.88150513399346,
   0.8813603855345206,
   0.8796997239785485,
   0.8628995748909527,
   0.8553689601369538,
   0.933631333532367,
   0.9267955151730277,
   0.8976902882963591,
   0.868908318888539,
   0.8568155677102007,
   0.8612003681374104,
   0.8623124696281521,
   0.8601045822762042,
   0.8583966234011755,
   0.8532985653526196,
   0.9360770441240855,
   0.9654639600581305,
   0.9650672803419056,
   0.9227437842541527,
   0.8634103073733801,
   0.8780394109319974,
   0.8811920698092427,
   0.8669565484953434,
   0.8545079971590418,
   0.8520

In [15]:
node_degrees = {file_name: [0] * len(time_axis) for file_name in file_names}

for (file1, file2), correlations in sliding_correlations.items():
    for t, corr in enumerate(correlations):
        if abs(corr) > correlation_threshold:
            node_degrees[file1][t] += 1
            node_degrees[file2][t] += 1
node_degrees = {file_name.split('_')[-1].split('.')[0]: degrees 
                        for file_name, degrees in node_degrees.items()}
node_degrees = {int(key): value for key, value in node_degrees.items()}
max_correlations = {int(key.split('_')[-1].split('.')[0]): values 
                             for key, values in max_correlations.items()}


In [16]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection
import numpy as np

# Setup the colormap
cmap = plt.cm.viridis
norm = plt.Normalize(vmin=np.nanmin([np.nanmin(v) for v in max_correlations.values() if np.isfinite(np.nanmin(v))]),
                     vmax=np.nanmax([np.nanmax(v) for v in max_correlations.values() if np.isfinite(np.nanmax(v))]))

fig, ax = plt.subplots(figsize=(45, 20))

# Create a circle for each node at each time point
for node, degrees_over_time in node_degrees.items():
    lat = latitude_dict[node]
    for time_idx, degree in enumerate(degrees_over_time):
        max_corr = max_correlations[node][time_idx]
        radius = degree * 0.01  # Adjust the factor for circle size as needed
        color = cmap(norm(max_corr)) if np.isfinite(max_corr) else 'white'
        # Add black edgecolor for better visibility
        circle = Circle((time_idx, lat), radius, color=color, ec='black', lw=0.1)  # lw is the linewidth of the edge
        ax.add_patch(circle)

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
fig.colorbar(sm, ax=ax)

# Set axis labels and limits
ax.set_xlabel('Time')
ax.set_ylabel('Latitude')
ax.set_xlim(0, len(time_axis))
ax.set_ylim(min(latitude_dict.values()), max(latitude_dict.values()))

plt.savefig('C:/Users/19606/Desktop/GMDandGIC/plot4.png')  # Save the figure to a file
plt.close()


In [17]:
# import matplotlib.pyplot as plt
# from matplotlib.patches import Circle
# from matplotlib.collections import PatchCollection
# import numpy as np

# # Setup the colormap
# cmap = plt.cm.viridis  # Choose a colormap that fits your aesthetic
# norm = plt.Normalize(vmin=np.nanmin([np.nanmin(v) for v in max_correlations.values() if np.isfinite(np.nanmin(v))]),
#                      vmax=np.nanmax([np.nanmax(v) for v in max_correlations.values() if np.isfinite(np.nanmax(v))]))

# fig, ax = plt.subplots(figsize=(45, 20))

# # Now create a circle for each node at each time point
# patches = []
# for node, degrees_over_time in node_degrees.items():
#     lat = latitude_dict[node]
#     for time_idx, degree in enumerate(degrees_over_time):
#         max_corr = max_correlations[node][time_idx]
#         radius = degree * 0.01  # Adjust the factor for circle size as needed

#         # Map the correlation value to a color
#         color = cmap(norm(max_corr)) if np.isfinite(max_corr) else 'white'  # Ignore infinite values

#         # Create a circle with the size and color mapped to the degree and correlation
#         circle = Circle((time_idx, lat), radius, color=color, ec=None)
#         patches.append(circle)

# # Create a collection with the circles
# pcol = PatchCollection(patches, match_original=True)
# ax.add_collection(pcol)

# # Add colorbar
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# sm.set_array([])
# fig.colorbar(sm, ax=ax)

# # Set axis labels and limits
# ax.set_xlabel('Time')
# ax.set_ylabel('Latitude')
# ax.set_xlim(0, len(time_axis)) 
# ax.set_ylim(min(latitude_dict.values()), max(latitude_dict.values()))

# plt.savefig('C:/Users/19606/Desktop/GMDandGIC/plot3.png')  # Save the figure to a file
# plt.close()
